In [ ]:
!pip install river

from river.drift import KSWIN
from river.datasets import Bikes
# from sdv.single_table import GaussianCopulaSynthesizer
# from sdv.metadata import SingleTableMetadata
import pandas as pd
from utils import plot_drift
from novelty import ephemeral, persistant, periodic

%load_ext autoreload
%autoreload 2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 87.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
train_split, test_split = 5000, 1000

In [3]:
# Step 1: Prepare Original Dataset
bikes = Bikes()
data_list = [x for x, y in bikes.take(train_split + test_split)]
train_data, test_data = data_list[:train_split], data_list[train_split:]
df_train, df_test = pd.DataFrame(train_data), pd.DataFrame(test_data)
df_all = pd.concat([df_train, df_test])

display(df_all)

Uncompressing into /root/river_data/Bikes


,moment,station,clouds,description,humidity,pressure,temperature,wind
0,2016-04-01 00:00:07,metro-canal-du-midi,75,light rain,81,1017.0,6.54,9.3
1,2016-04-01 00:00:16,place-des-carmes,75,light rain,81,1017.0,6.54,9.3
2,2016-04-01 00:00:35,place-des-carmes,75,light rain,81,1017.0,6.54,9.3
3,2016-04-01 00:04:50,place-esquirol,75,light rain,81,1017.0,6.54,9.3
4,2016-04-01 00:05:29,place-esquirol,75,light rain,81,1017.0,6.54,9.3
...,...,...,...,...,...,...,...,...
995,2016-04-04 14:24:03,pomme,90,overcast clouds,63,1004.0,15.42,5.7
996,2016-04-04 14:24:41,place-jeanne-darc,90,overcast clouds,63,1004.0,15.42,5.7
997,2016-04-04 14:24:43,pomme,90,overcast clouds,63,1004.0,15.42,5.7
998,2016-04-04 14:25:48,place-esquirol,90,overcast clouds,63,1004.0,15.42,5.7


In [4]:
feature_name = 'humidity'

In [6]:
plot_drift(feature_name, df_train, df_test)

In [12]:
def detect_drift(feature_name, train_data, test_data):
    # Step 4: Initialize Drift Detector
    drift_detector =KSWIN(alpha=0.0001, seed=42)
    # Step 5: Track Drift and Store Detected Points
    drift_points = []  # To store the indices where drift is detected

    for value in train_data[feature_name]:
        drift_detector.update(value)

    # Step 6: Feed Synthetic Data into Drift Detector (Comparing Only to Synthetic Past)
    for i, value in enumerate(test_data[feature_name]):
        drift_detector.update(value)
        if drift_detector.drift_detected:
            print(f"🚨 Drift detected at sample {i+1}, value: {value}")

            drift_points.append(i)

    return drift_points

In [13]:
drift_points = detect_drift(feature_name, df_train, df_test)
plot_drift(feature_name, df_train, df_test, drift_points)

🚨 Drift detected at sample 92, value: 76
🚨 Drift detected at sample 192, value: 87
🚨 Drift detected at sample 306, value: 81
🚨 Drift detected at sample 495, value: 81
🚨 Drift detected at sample 595, value: 71
🚨 Drift detected at sample 731, value: 62
🚨 Drift detected at sample 831, value: 59
🚨 Drift detected at sample 931, value: 67


In [14]:
df_ephemeral = ephemeral(df_test, feature_name, 200, 300, lambda x: x + 30)
drift_points = detect_drift(feature_name, df_train, df_ephemeral)
plot_drift(feature_name, df_train, df_ephemeral, drift_points)

🚨 Drift detected at sample 92, value: 76
🚨 Drift detected at sample 192, value: 87
🚨 Drift detected at sample 306, value: 111
🚨 Drift detected at sample 495, value: 111
🚨 Drift detected at sample 595, value: 71
🚨 Drift detected at sample 731, value: 62
🚨 Drift detected at sample 831, value: 59
🚨 Drift detected at sample 931, value: 67


In [18]:
df_persistant = persistant(df_test, feature_name, 200, lambda x: x + 30)
drift_points = detect_drift(feature_name, df_train, df_persistant)
plot_drift(feature_name, df_train, df_persistant, drift_points)

🚨 Drift detected at sample 92, value: 76
🚨 Drift detected at sample 192, value: 87
🚨 Drift detected at sample 306, value: 111
🚨 Drift detected at sample 495, value: 111
🚨 Drift detected at sample 595, value: 101
🚨 Drift detected at sample 731, value: 92
🚨 Drift detected at sample 831, value: 89
🚨 Drift detected at sample 931, value: 97


In [17]:
df_persistant = persistant(df_test, feature_name, 200, lambda x: x + 30, after=False)
drift_points = detect_drift(feature_name, df_train, df_persistant)
plot_drift(feature_name, df_train, df_persistant, drift_points)

🚨 Drift detected at sample 92, value: 106
🚨 Drift detected at sample 192, value: 117
🚨 Drift detected at sample 306, value: 81
🚨 Drift detected at sample 495, value: 81
🚨 Drift detected at sample 595, value: 71
🚨 Drift detected at sample 731, value: 62
🚨 Drift detected at sample 831, value: 59
🚨 Drift detected at sample 931, value: 67


In [19]:
df_periodic = periodic(df_test, feature_name, 200, 200, 300, lambda x: x + 30)
drift_points = detect_drift(feature_name, df_train, df_periodic)
plot_drift(feature_name, df_train, df_periodic, drift_points)

🚨 Drift detected at sample 92, value: 76
🚨 Drift detected at sample 192, value: 87
🚨 Drift detected at sample 306, value: 111
🚨 Drift detected at sample 418, value: 87
🚨 Drift detected at sample 518, value: 111
🚨 Drift detected at sample 618, value: 96
🚨 Drift detected at sample 718, value: 62
🚨 Drift detected at sample 818, value: 89
🚨 Drift detected at sample 918, value: 97


In [15]:
df_persistant = persistant(df_test, feature_name, 200, lambda x: x[feature_name] + x.name // 10, on_row=True)
drift_points = detect_drift(feature_name, df_train, df_persistant)
plot_drift(feature_name, df_train, df_persistant, drift_points)

🚨 Drift detected at sample 92, value: 76
🚨 Drift detected at sample 192, value: 87
🚨 Drift detected at sample 296, value: 110
🚨 Drift detected at sample 396, value: 126
🚨 Drift detected at sample 539, value: 134
🚨 Drift detected at sample 654, value: 132
🚨 Drift detected at sample 761, value: 138
🚨 Drift detected at sample 861, value: 145
🚨 Drift detected at sample 961, value: 163
